In [1]:
from google.colab import files
uploaded = files.upload()

Saving cats_vs_dogs_dataset.zip to cats_vs_dogs_dataset.zip


In [4]:
import zipfile
import os
zip_path = list(uploaded.keys())[0] # automatically get the uploaded filename
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
  zip_ref.extractall('/content/') # extract to /content
# Confirm extraction
os.listdir('/content')

['.config', 'cats_vs_dogs_dataset', 'cats_vs_dogs_dataset.zip', 'sample_data']

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [7]:
dataset_path = '/content/cats_vs_dogs_dataset'
IMG_SIZE = 128
BATCH_SIZE = 32

In [9]:
datagen = ImageDataGenerator(rescale = 1./255, validation_split = 0.2)
train_data = datagen.flow_from_directory(dataset_path,
                                         target_size = (IMG_SIZE, IMG_SIZE),
                                         batch_size = BATCH_SIZE,
                                         class_mode = 'binary',
                                         subset = 'training')

val_data = datagen.flow_from_directory(dataset_path,
                                         target_size = (IMG_SIZE, IMG_SIZE),
                                         batch_size = BATCH_SIZE,
                                         class_mode = 'binary',
                                         subset = 'validation')

Found 17294 images belonging to 2 classes.
Found 4322 images belonging to 2 classes.


In [12]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation = 'relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation = 'relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation = 'relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation = 'sigmoid')
])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
history = model.fit(train_data, validation_data = val_data, epochs = 5)

Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


541/541 ━━━━━━━━━━━━━━━━━━━━ 549s 1s/step - accuracy: 0.5842 - loss: 0.6869 - val_accuracy: 0.6518 - val_loss: 0.6208
Epoch 2/5
541/541 ━━━━━━━━━━━━━━━━━━━━ 515s 953ms/step - accuracy: 0.7113 - loss: 0.5659 - val_accuracy: 0.7550 - val_loss: 0.5060
Epoch 3/5
541/541 ━━━━━━━━━━━━━━━━━━━━ 505s 934ms/step - accuracy: 0.7716 - loss: 0.4765 - val_accuracy: 0.7918 - val_loss: 0.4542
Epoch 4/5
541/541 ━━━━━━━━━━━━━━━━━━━━ 507s 937ms/step - accuracy: 0.8054 - loss: 0.4216 - val_accuracy: 0.8077 - val_loss: 0.4278
Epoch 5/5
541/541 ━━━━━━━━━━━━━━━━━━━━ 508s 940ms/step - accuracy: 0.8385 - loss: 0.3635 - val_accuracy: 0.7932 - val_loss: 0.4791


In [ ]:
from tensorflow.keras.preprocessing import image

uploaded_test = files.upload()
for filename in uploaded_test.keys():
  img_path = '/content/' + filename
  img = image.load_img(img_path, target_size = (IMG_SIZE, IMG_SIZE))
  img_array = image.img_to_array(img)/255.0
  img_array = np.expand_dims(img_array, axis = 0)

  prediction = model.predict(img_array)
  label = 'Dog🦮' if prediction[0][0] > 0.5 else 'Cat🐱‍👤'